RESULTADOS

In [13]:
from models import Models
from losses import *
import numpy as np

import cv2
import scipy
import glob
import matplotlib.pyplot as plt

import imageio
from time import time, sleep

from keras.backend.tensorflow_backend import set_session
from keras import backend as K
import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = False  # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(sess)  # set this TensorFlow session as the default session for Keras

%load_ext autoreload
%autoreload 2 

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16676190643867356226,
 name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 14950894760617235523
 physical_device_desc: "device: XLA_CPU device"]

LINE OF HORIZONT FITTING

In [15]:

class Line_of_horizont_fitting:

    line=[]

    def inputNormalized(self, image, img_w, img_h):
        image=self.resize_image(image, img_w, img_h)
        #image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image=image/255.0
        return image

    def resize_image(self, image, img_w, img_h):
        image = cv2.resize(image,(img_w,img_h))
        return image

    def plot_binary_image(self, image):
        image=image*255
        binary_img = np.squeeze(image, axis=2)
        plt.imshow(binary_img)
        plt.show()

    def get_binary_image(self, image, treshold):
        image = cv2.threshold(image,treshold,1,cv2.THRESH_BINARY)
        return image[1]

    def binary_edge_detection(self, image):
        edges = image - scipy.ndimage.morphology.binary_dilation(image)
        return edges

    def median_blur(self, img, kernel_size):
        """Applies a Gaussian Noise kernel"""
        return cv2.blur(img, (kernel_size, kernel_size))

    def hough_lines(self, img, rho, theta, threshold, min_line_len, max_line_gap):  
        lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
        return lines

    def binary2gray(self, image):
        image = np.uint8(255 * image)
        return image

    def Collect_points(self, lines):

        # interpolation & collecting points for RANSAC
        points=[]
        for line in lines:
            new_point = np.array([[int(line[0]),int(line[1])]])
            points.append(new_point)
            new_point = np.array([[int(line[2]),int(line[3])]])
            points.append(new_point)

        return points

    def smoothing(self, lines, pre_frame=10):
        # collect frames & print average line
        lines = np.squeeze(lines)
        avg_line = np.array([0.0,0.0,0.0,0.0])

        for ii,line in enumerate(reversed(lines)):
            if ii == pre_frame:
                break
            avg_line += line
        avg_line = avg_line / pre_frame

        return avg_line

    def getLineImage(self, image,label,fit_line, width, height):
        height,width,_=image.shape
        imageOUT = cv2.bitwise_or(image,label)
        cv2.line(imageOUT, (int(fit_line[2]-fit_line[0]*width), int(fit_line[3]-fit_line[1]*width)), (int(fit_line[2]+fit_line[0]*width), int(fit_line[3]+fit_line[1]*width)), (255, 0, 255), 12)

    def predict_segmentation(self, image, model):
        predict = model.predict(image[None,...])

        return predict[0]
    
    def horizont_line_from_binary_image(self, image, average_n_frame=1, rho = 2, theta = np.pi/180, threshold = 20, min_line_length = 20, 
                                        max_line_gap = 5):
        output = self.binary_edge_detection(image)
        output = self.binary2gray(output)

        rho = rho# distance resolution in pixels of the Hough grid
        theta = theta # angular resolution in radians of the Hough grid
        threshold = threshold     # minimum number of votes (intersections in Hough grid cell)
        min_line_length = min_line_length #minimum number f pixels making up a line
        max_line_gap = 5    # maximum gap in pixels between connectable line segments
        lines=self.hough_lines(output, rho, theta, threshold, min_line_length, max_line_gap)
        line_arr = np.squeeze(lines)
        points = self.Collect_points(line_arr)

        if(len(points)<2):
            points= line_arr.reshape(lines.shape[0]*2,2)

        if(len(points)>2):
            fit_line = cv2.fitLine(np.float32(points), cv2.DIST_HUBER, 1, 0.001, 0.001)
            #self.line.append(fit_line)

            #if len(self.line) > 10:
            #    fit_line = self.smoothing(self.line, average_n_frame)

        return fit_line
        

    def horizont_line_pipeline(self, image, model, img_w, img_h, average_n_frame, kernel_median_blur=50, predict_treshold=0.5,
                               rho = 2, theta = np.pi/180, threshold = 20, min_line_length = 20, max_line_gap = 5):
        or_image=image

        or_height, or_width, or_depth = or_image.shape
        image=self.inputNormalized(or_image,img_w,img_h)
        predict_segmentation = self.predict_segmentation(image,model)
        predict = self.median_blur(predict_segmentation,kernel_median_blur)
        predict = self.get_binary_image(predict,predict_treshold)
        predict = self.resize_image(predict, or_width, or_height)
        output = self.binary_edge_detection(predict)
        output = self.binary2gray(output)

        rho = rho# distance resolution in pixels of the Hough grid
        theta = theta # angular resolution in radians of the Hough grid
        threshold = threshold     # minimum number of votes (intersections in Hough grid cell)
        min_line_length = min_line_length #minimum number f pixels making up a line
        max_line_gap = 5    # maximum gap in pixels between connectable line segments
        lines=self.hough_lines(output, rho, theta, threshold, min_line_length, max_line_gap)
        line_arr = np.squeeze(lines)
        points = self.Collect_points(line_arr)

        if(len(points)<2):
            points= line_arr.reshape(lines.shape[0]*2,2)

        if(len(points)>2):
            fit_line = cv2.fitLine(np.float32(points), cv2.DIST_HUBER, 1, 0.001, 0.001)
            #self.line.append(fit_line)

            #if len(self.line) > 10:
            #    fit_line = self.smoothing(self.line, average_n_frame)

        return fit_line, self.resize_image(predict_segmentation, or_width, or_height)
    
    def horizont_line_pipeline_verbose(self, image, model, img_w, img_h, average_n_frame, kernel_median_blur=50, predict_treshold=0.5, rho = 2, theta = np.pi/180, threshold = 20, min_line_length = 20, max_line_gap = 5):
        or_image=image

        or_height, or_width, or_depth = or_image.shape
        image=self.inputNormalized(or_image,img_w,img_h)
        predict_segmentation = self.predict_segmentation(image,model)
        predict = self.median_blur(predict_segmentation,kernel_median_blur)
        predict = self.get_binary_image(predict,predict_treshold)
        predict = self.resize_image(predict, or_width, or_height)
        output = self.binary_edge_detection(predict)
        output = self.binary2gray(output)

        rho = rho# distance resolution in pixels of the Hough grid
        theta = theta # angular resolution in radians of the Hough grid
        threshold = threshold     # minimum number of votes (intersections in Hough grid cell)
        min_line_length = min_line_length #minimum number f pixels making up a line
        max_line_gap = 5    # maximum gap in pixels between connectable line segments
        lines=self.hough_lines(output, rho, theta, threshold, min_line_length, max_line_gap)
        line_arr = np.squeeze(lines)
        points = self.Collect_points(line_arr)

        if(len(points)<2):
            points= line_arr.reshape(lines.shape[0]*2,2)

        if(len(points)>2):
            fit_line = cv2.fitLine(np.float32(points), cv2.DIST_HUBER, 1, 0.001, 0.001)
            #self.line.append(fit_line)

            #if len(self.line) > 10:
            #    fit_line = self.smoothing(self.line, average_n_frame)
        
        pred_visual = predict_segmentation*255
        pred_visual= np.uint8(np.concatenate((predict_segmentation,predict_segmentation,pred_visual),axis=2))
        return fit_line, self.resize_image(predict_segmentation, or_width, or_height), self.resize_image(or_image, img_w, img_h), self.resize_image(pred_visual, img_w, img_h)

UTILS

In [16]:

class Utils:
    @staticmethod
    def median_accuracy_line_of_horizont(x, y, model, inp_w, inp_h, steps=1, visualization=False):
        avg_distance=[]
        max_distance=[]
        
        line_of_horizont=Line_of_horizont_fitting()
        for label, img in zip(y, x):

            image = np.uint8(255 * img)
            
            label_med=line_of_horizont.median_blur(label,5)
            label_med=line_of_horizont.get_binary_image(label_med, 0.5)
            fit_line= line_of_horizont.horizont_line_from_binary_image(label_med)

            label=line_of_horizont.get_binary_image(label, 0.5)
            height,width=label.shape
            label_line = np.zeros([height,width], dtype = "uint8")
            cv2.line(label_line, (int(fit_line[2]-fit_line[0]*width), 
                                             int(fit_line[3]-fit_line[1]*width)), 
                     (int(fit_line[2]+fit_line[0]*width), 
                      int(fit_line[3]+fit_line[1]*width)), (255, 255, 255), 1)

            image_pred = line_of_horizont.resize_image(img, inp_w, inp_h)
            pred=line_of_horizont.predict_segmentation(image_pred, model)
            pred=line_of_horizont.get_binary_image(pred, 0.5)
            pred=line_of_horizont.resize_image(pred, width, height)

            fit_line, predict=line_of_horizont.horizont_line_pipeline(image, model, inp_w, inp_h, steps, 5)

            pred_line = np.zeros([height,width], dtype = "uint8")
            cv2.line(pred_line, (int(fit_line[2]-fit_line[0]*width), 
                                             int(fit_line[3]-fit_line[1]*width)), 
                     (int(fit_line[2]+fit_line[0]*width), 
                      int(fit_line[3]+fit_line[1]*width)), (255, 255, 255), 1)

            distance=[]
            for j in range (width):
                for i in range (height):
                    if(label_line[i,j]==255):
                        y1=i
                    if(pred_line[i,j]==255):
                        y2=i
                distance.append(abs(y1-y2))

            avg_y= int((y1+y2)/2)
            
            avg_distance.append(np.mean(distance)/width)
            max_distance.append(max(distance))
            
            if(visualization):
                print("avg_distance: ", (np.mean(distance)/width)," - max_distance: ", (max(distance)))
                plt.imshow(label_line)
                plt.show()
                plt.imshow(pred_line)
                plt.show()
        
        return avg_distance, max_distance
    
    
    @staticmethod
    def accuracy_on_line_of_horizont_area(x, y, model, inp_w, inp_h, steps=1, visualization=False):
        recall_list=[]
        precision_list=[]
        specificity_list=[]
        accuracy_list=[]
        f1score_list=[]

        line_of_horizont=Line_of_horizont_fitting()
        for label, img in zip(y, x):

            image = np.uint8(255 * img)
            label=line_of_horizont.get_binary_image(label, 0.5)
            height,width=label.shape

            image_pred = line_of_horizont.resize_image(img, inp_w, inp_h)
            pred=line_of_horizont.predict_segmentation(image_pred, model)
            pred=line_of_horizont.get_binary_image(pred, 0.5)
            pred=line_of_horizont.resize_image(pred, width, height)

            fit_line, predict=line_of_horizont.horizont_line_pipeline(image, model, inp_w, inp_h, steps)

            line_annotation_image = np.zeros([height,width], dtype = "uint8")
            cv2.line(line_annotation_image, (int(fit_line[2]-fit_line[0]*width), 
                                             int(fit_line[3]-fit_line[1]*width)), 
                     (int(fit_line[2]+fit_line[0]*width), 
                      int(fit_line[3]+fit_line[1]*width)), (255, 255, 255), 1)

            for i in range (height):
                if(label[i,0]==1):
                    y1=i
                    break

            for i in range (height):
                if(label[i,width-1]==1):
                    y2=i
                    break

            avg_y= int((y1+y2)/2)

            annotation_image = label[avg_y-100:avg_y+100, 0:width]
            pred_image = pred[avg_y-100:avg_y+100, 0:width]

            label=annotation_image
            pred=pred_image

            True_neg=len(np.where((label==0)&(pred==0))[0])
            False_neg=len(np.where((label==1)&(pred==0))[0])
            True_pos=len(np.where((label==1)&(pred==1))[0])
            False_pos=len(np.where((label==0)&(pred==1))[0])
            precision=True_pos/(True_pos+False_pos)
            recall=True_pos/(True_pos+False_neg)
            specificity=1-(True_neg/(True_neg+False_pos))
            accuracy=(True_pos+True_neg)/(True_pos+True_neg+False_pos+False_neg)
            f1score=2*((precision*recall)/(precision+recall))

            recall_list.append(recall)
            precision_list.append(precision)
            specificity_list.append(specificity)
            accuracy_list.append(accuracy)
            f1score_list.append(f1score)

            if(visualization):
                print("Recall: ", recall," - Precision: ", precision, " - Specificity: ", specificity, " - Accuracy: ", 
                      accuracy, " - F1score: ", f1score)
                plt.imshow(label)
                plt.show()
                plt.imshow(pred)
                plt.show()

        return recall_list, precision_list, specificity_list, accuracy_list, f1score_list
    
    
    @staticmethod
    def accuracy_on_images(x, y, model, inp_w, inp_h, steps=1, visualization=False):
        recall_list=[]
        precision_list=[]
        specificity_list=[]
        accuracy_list=[]
        f1score_list=[]
        
        line_of_horizont=Line_of_horizont_fitting()
        for label, img in zip(y, x):

            label=line_of_horizont.get_binary_image(label, 0.5)
            height,width=label.shape
            image_pred = line_of_horizont.resize_image(img, 160, 160)
            pred=line_of_horizont.predict_segmentation(image_pred, model)
            pred=line_of_horizont.get_binary_image(pred, 0.5)
            pred=line_of_horizont.resize_image(pred, width, height)

            True_neg=len(np.where((label==0)&(pred==0))[0])
            False_neg=len(np.where((label==1)&(pred==0))[0])
            True_pos=len(np.where((label==1)&(pred==1))[0])
            False_pos=len(np.where((label==0)&(pred==1))[0])
            precision=True_pos/(True_pos+False_pos)
            recall=True_pos/(True_pos+False_neg)
            specificity=1-(True_neg/(True_neg+False_pos))
            accuracy=(True_pos+True_neg)/(True_pos+True_neg+False_pos+False_neg)
            f1score=2*((precision*recall)/(precision+recall))
            
            recall_list.append(recall)
            precision_list.append(precision)
            specificity_list.append(specificity)
            accuracy_list.append(accuracy)
            f1score_list.append(f1score)
            
            if(visualization):
                print("Recall: ", recall," - Precision: ", precision, " - Specificity: ", specificity, " - Accuracy: ", 
                      accuracy, " - F1score: ", f1score)
                plt.imshow(label)
                plt.show()
                plt.imshow(pred)
                plt.show()
            
        return recall_list, precision_list, specificity_list, accuracy_list, f1score_list
            
    
    
    @staticmethod
    def test_speed_from_video(filename, model, inp_w, inp_h, n_iteration, steps=1):
        lineofhorizont = Line_of_horizont_fitting()
        reader = imageio.get_reader(filename,  'ffmpeg')
        fps = reader.get_meta_data()['fps']
        n_steps=0
        frame_to_discard = 10
        now=time()
        for i in range(n_iteration):
            if i == frame_to_discard:
                start_time=now
            if i > frame_to_discard:
                #n_steps+=1
                elapsed_time = now - start_time
                #print(n_steps, elapsed_time)
            or_image=reader.get_data(i)
            or_height, or_width, or_depth = or_image.shape

            fit_line, predict=lineofhorizont.horizont_line_pipeline(or_image, model, inp_w, inp_h, steps)

            predict = predict.reshape(or_height,or_width,1)
            predict1 = predict*255
            predict= np.uint8(np.concatenate((predict,predict,predict1),axis=2))
            imageOUT = cv2.bitwise_or(or_image,predict)
            imageOUT=cv2.line(imageOUT, (int(fit_line[2]-fit_line[0]*or_height), int(fit_line[3]-fit_line[1]*or_width)), 
                              (int(fit_line[2]+fit_line[0]*or_height), int(fit_line[3]+fit_line[1]*or_width)), 
                              (255, 0, 255), 5)
            now=time()
            
        reader.close()
        return ((i-frame_to_discard))/elapsed_time
    
    @staticmethod
    def test_speed_from_video_v2(reader, model, inp_w, inp_h, n_iteration, steps=1):
        n_steps=0
        frame_to_discard = 10
        now=time()
        lineofhorizont = Line_of_horizont_fitting()
        for i in range(n_iteration):
            if i == frame_to_discard:
                start_time=now
            if i > frame_to_discard:
                #n_steps+=1
                elapsed_time = now - start_time
                #print(n_steps, elapsed_time)
            or_image=reader.get_data(i)
            or_height, or_width, or_depth = or_image.shape

            fit_line, predict=lineofhorizont.horizont_line_pipeline(or_image, model, inp_w, inp_h, steps)

            predict = predict.reshape(or_height,or_width,1)
            predict1 = predict*255
            predict= np.uint8(np.concatenate((predict,predict,predict1),axis=2))
            imageOUT = cv2.bitwise_or(or_image,predict)
            imageOUT=cv2.line(imageOUT, (int(fit_line[2]-fit_line[0]*or_height), int(fit_line[3]-fit_line[1]*or_width)), 
                              (int(fit_line[2]+fit_line[0]*or_height), int(fit_line[3]+fit_line[1]*or_width)), 
                              (255, 0, 255), 5)
            now=time()

        return ((i-frame_to_discard))/elapsed_time
    
    @staticmethod
    def test_from_video(filename, model, inp_w, inp_h, n_iteration, steps=1):
        lineofhorizont = Line_of_horizont_fitting()
        reader = imageio.get_reader(filename,  'ffmpeg')
        fps = reader.get_meta_data()['fps']
        now=time()
        start_time=now
        for i in range(n_iteration):
            elapsed_time = now - start_time
            print(i, elapsed_time)
            or_image=reader.get_data(i)
            plt.imshow(or_image)
            plt.show()
            or_height, or_width, or_depth = or_image.shape

            fit_line, predict=lineofhorizont.horizont_line_pipeline(or_image, model, inp_w, inp_h, steps)

            predict = predict.reshape(or_height,or_width,1)
            predict1 = predict*255
            predict= np.uint8(np.concatenate((predict,predict,predict1),axis=2))
            imageOUT = cv2.bitwise_or(or_image,predict)
            imageOUT=cv2.line(imageOUT, (int(fit_line[2]-fit_line[0]*or_height), int(fit_line[3]-fit_line[1]*or_width)), 
                              (int(fit_line[2]+fit_line[0]*or_height), int(fit_line[3]+fit_line[1]*or_width)), 
                              (255, 0, 255), 5)
            now=time()
            plt.imshow(imageOUT)
            plt.show()
        reader.close()

    @staticmethod
    def test_from_folder(path, model, inp_w, inp_h, steps=1):
        lineofhorizont = Line_of_horizont_fitting()
        path_images=glob.glob(path)
        images=[]
        now=time()
        start_time=now
        for path_img in path_images:
            elapsed_time = now - start_time
            print(elapsed_time)
            or_image=cv2.imread(path_img)
            or_image=cv2.cvtColor(or_image, cv2.COLOR_BGR2RGB)
            or_height, or_width, or_depth = or_image.shape

            fit_line, predict, img_inp_or, pred_inp_or=lineofhorizont.horizont_line_pipeline_verbose(or_image, model, inp_w, inp_h, steps)

            predict = predict.reshape(or_height,or_width,1)
            predict1 = predict*255
            predict= np.uint8(np.concatenate((predict,predict,predict1),axis=2))
            imageOUT = cv2.bitwise_or(or_image,predict)
            
            #(x0-m*vx[0], y0-m*vy[0]), (x0+m*vx[0], y0+m*vy[0])
            
            W = or_width 
            H = or_height
            
            x0 = (int(fit_line[2]-(W*fit_line[0])))
            x1 = (int(fit_line[2]+(W*fit_line[0])))
            y0 = (int(fit_line[3]-(H*fit_line[1])))
            y1 = (int(fit_line[3]+(H*fit_line[1])))
            
            imageOUT=cv2.line(imageOUT, (x0,y0), (x1,y1), (255, 0, 255), 5)
            now=time()
        
            yield path_img, imageOUT, predict, img_inp_or, pred_inp_or


In [17]:
path_models='./models_saved/unet160--largeMobileNet--dice_coef_loss'
img_w=160
img_h=160
# Para custom objects
def f1(y_true, y_pred):
    return 1

model_class = Models()


model1 = model_class.load_model(path_models)

#model1=model_class.load_model(path_models+'unet160--mediumBN--dice_coef_loss')

ValueError: Unable to restore custom object of type _tf_keras_metric currently. Please make sure that the layer implements `get_config`and `from_config` when saving. In addition, please use the `custom_objects` arg when calling `load_model()`.

In [45]:
import matplotlib.pyplot as plt
import cv2
path_images = "./video_input/Test2/*"
path_2_save = "./video_input/Test2_result/"
for path_img, imageOUT, predict, img_inp_or, pred_inp_or in Utils.test_from_folder(path_images, model1, img_w, img_h, steps=300):
    name= path_img[path_img.rfind("/") + 1::]
    print(name)
    cv2.imwrite(path_2_save+"prediction-"+name, cv2.cvtColor(imageOUT, cv2.COLOR_RGB2BGR))
    cv2.imwrite(path_2_save+"mask_prediction-"+name, cv2.cvtColor(predict, cv2.COLOR_RGB2BGR))
    cv2.imwrite(path_2_save+"image_inpNN_or_size-"+name, cv2.cvtColor(img_inp_or, cv2.COLOR_RGB2BGR))
    cv2.imwrite(path_2_save+"mask_prediction_or_size-"+name, cv2.cvtColor(pred_inp_or, cv2.COLOR_RGB2BGR))

NameError: name 'model1' is not defined

In [ ]:
filename = "video_input/prophr3.mp4"

Utils.test_from_video(filename, model1, img_w, img_h, 100)

In [ ]:
filename = "video_input/lakegarda-may-9-prop-8.avi"#prophr3.mp4"

Utils.test_speed_from_video(filename, model1, img_w, img_h, 100)

In [ ]:
from dataset import Dataset
path = 'Test/'
img_w = 1280
img_h = 720
n_labels = 2 #6
dataset = Dataset(path, img_w, img_h, n_labels)
x, y = dataset.createDataset(augmentation=False, prob1=False)

In [ ]:
model_class = Models()
path_models='./models_saved/Final_2/'
model_path_name= ['unet160--small--dice_coef_loss',
                  'unet160--smallBN--dice_coef_loss',
                  'unet160--medium--dice_coef_loss',
                  'unet160--mediumBN--dice_coef_loss',
                  'unet160--mediumBN_mobilenetv1--dice_coef_loss',
                  ]

for models_name in model_path_name:
    model1=model_class.load_model(path_models+models_name)
    recall_list, precision_list, specificity_list, accuracy_list, f1score_list = Utils.accuracy_on_images(x, y, model1, 160, 160, 1, False)
    print('*' * 30)
    print(models_name)
    print('N PARAMETERS =', model1.count_params())
    print("RECALL =", sum(recall_list) / float(len(recall_list)))
    print("PRECISION =", sum(precision_list) / float(len(precision_list)))
    print("SPECIFICITY =", sum(specificity_list) / float(len(specificity_list)))
    print("ACCURACY =", sum(accuracy_list) / float(len(accuracy_list)))
    print("F1SCORE =", sum(f1score_list) / float(len(f1score_list)))
    print('*' * 30)
    
    K.clear_session()

In [ ]:
model_class = Models()
path_models='./models_saved/Final_2/'
model_path_name= ['unet160--small--dice_coef_loss',
                  'unet160--smallBN--dice_coef_loss',
                  'unet160--medium--dice_coef_loss',
                  'unet160--mediumBN--dice_coef_loss',
                  'unet160--mediumBN_mobilenetv1--dice_coef_loss',
                  ]

for models_name in model_path_name:
    model1=model_class.load_model(path_models+models_name)
    recall_list, precision_list, specificity_list, accuracy_list, f1score_list = Utils.accuracy_on_line_of_horizont_area(x, y, model1, 160, 160, 1, True)
    print('*' * 30)
    print(models_name)
    print('N PARAMETERS =', model1.count_params())
    print("PRECISION =", sum(precision_list) / float(len(precision_list)))
    print("RECALL =", sum(recall_list) / float(len(recall_list)))
    #print("SPECIFICITY =", sum(specificity_list) / float(len(specificity_list)))
    print("ACCURACY =", sum(accuracy_list) / float(len(accuracy_list)))
    print("F1SCORE =", sum(f1score_list) / float(len(f1score_list)))
    print('*' * 30)

    K.clear_session()

In [ ]:
model_class = Models()
path_models='./models_saved/Final_2/'
model_path_name= ['unet160--small--dice_coef_loss',
                  'unet160--smallBN--dice_coef_loss',
                  'unet160--medium--dice_coef_loss',
                  'unet160--mediumBN--dice_coef_loss',
                  'unet160--mediumBN_mobilenetv1--dice_coef_loss',
                  ]

max_distances_array = []
names_array = []

for models_name in model_path_name:
    model1=model_class.load_model(path_models+models_name)
    avg_distance, max_distance = Utils.median_accuracy_line_of_horizont(x, y, model1, 160, 160, 1, False)
    print('*' * 30)
    print("-- ", models_name, " --")
    print("Mean Max error :", np.mean(max_distance))
    print("Median Max error :", np.median(max_distance))
    print("Standard Deviation :", np.std(max_distance))
    print("Quantile 0.25-0.75:", np.quantile(max_distance, 0.25), " - ", np.quantile(max_distance, 0.75))
    print("Average error :", np.mean(avg_distance))
    print("Max error :", np.max(max_distance))
    print('*' * 30)
    
    names_array.append(models_name)
    max_distances_array.append(max_distance)
    
    K.clear_session()

In [ ]:
names_array = ["Half-Conv 160x160", "Half-Conv BN 160x160", "Full 160x160", "Full BN 160x160", "Full mobile-net-v1-layer 160x160"]

In [ ]:
fig1, (ax1, ax2) = plt.subplots(1, 2, sharey=True, gridspec_kw={'width_ratios': [10, 10]})

ax1.boxplot(max_distances_array, vert=False, labels=names_array)
ax1.set_xlim(0,30)

ax2.boxplot(max_distances_array, vert=False, labels=names_array)
ax2.set_xlim(60,100)

ax1.spines['right'].set_visible(False)
ax2.spines['left'].set_visible(False)
ax1.yaxis.tick_left()
ax1.tick_params(labelright='off')
ax2.yaxis.tick_right()
ax2.tick_params(labelright='off')

d = .015 # how big to make the diagonal lines in axes coordinates
# arguments to pass plot, just so we don't keep repeating them
kwargs = dict(transform=ax1.transAxes, color='k', clip_on=False)
ax1.plot((1-d,1+d), (-d,+d), **kwargs)
ax1.plot((1-d,1+d),(1-d,1+d), **kwargs)

kwargs.update(transform=ax2.transAxes)  # switch to the bottom axes
ax2.plot((-d,+d), (1-d,1+d), **kwargs)
ax2.plot((-d,+d), (-d,+d), **kwargs)

fig1.subplots_adjust(top=0.9)

In [ ]:
import matplotlib.pyplot as plt
fig2, ax2 = plt.boxplot(max_distances_array, vert=False, labels=names_array)

In [ ]:
!cd models_saved/history/

In [ ]:
import pickle
import os
import matplotlib.pyplot as plt
history_path=['models_saved/history/unet160--history--largeBN--binary_crossentropy',
              'models_saved/history/unet160--history--largeBN--mae', 
              'models_saved/history/unet160--history--largeBN--mse',
              'models_saved/history/unet160--history--largeBN--dice_coef_loss', ]

historyes = []
for path in history_path:
    with (open(path+"", "rb")) as openfile:
        while True:
            try:
                historyes.append(pickle.load(openfile))
            except EOFError:
                break

In [ ]:
# list all data in history
print(historyes[0].keys())
# summarize history for loss
for history in historyes:
    plt.plot(history['val_dice_coef'])
plt.title('model accuracy')
plt.ylabel('Dice/F1 metric')
plt.xlabel('epoch')
plt.legend(['binary_crossentropy_loss',
            'mean_absolute_error_loss',
            'mean_squared_error_loss',
            'dice_coef_loss'], loc='lower right')
plt.axis([0,200,0.9,1])
plt.show()

In [8]:
import cv2
import numpy as np
file = "ter1_1-annotation.png"
img=cv2.imread(file,0)
hist = np.histogram(img.flatten(),256,[0,256])[0]

In [10]:
print(hist)

[ 917031 1156569       0       0       0       0       0       0       0
       0       0       0       0       0       0       0       0       0
       0       0       0       0       0       0       0       0       0
       0       0       0       0       0       0       0       0       0
       0       0       0       0       0       0       0       0       0
       0       0       0       0       0       0       0       0       0
       0       0       0       0       0       0       0       0       0
       0       0       0       0       0       0       0       0       0
       0       0       0       0       0       0       0       0       0
       0       0       0       0       0       0       0       0       0
       0       0       0       0       0       0       0       0       0
       0       0       0       0       0       0       0       0       0
       0       0       0       0       0       0       0       0       0
       0       0       0       0       0       0   